신경망은 layer과 module로 구성

torch.nn - 신경망을 구성하는데 필요한 모든 구성 요소를 제공

In [1]:
# FashionMNIST 이미지 분류 신경망 구성

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
# define Class

class NeuralNetwork(nn.Module):
    def __init__(self): # 신경망 계층 초기화
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x): # forward 연산
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# 인스턴스 생성

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


In [7]:
# Layer

input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
# 28x28의 2D이미지를 784 픽셀값을 갖는 연속 배열로 변환

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
# 선형 변환

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
# 비선형 근사 ReLU함수 적용

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1056,  0.2080,  0.0040, -0.6640, -0.2360,  1.0676, -0.4269,  0.5721,
          0.1048,  0.3157, -0.6253,  0.1254,  0.2161, -0.0073, -0.3233,  0.1460,
          0.0118,  0.3129, -0.0148, -0.1486],
        [ 0.0859,  0.0677, -0.0306, -0.4227, -0.4158,  1.1362, -0.2528, -0.1066,
          0.1357,  0.3376, -0.7674, -0.0821, -0.0613,  0.1660, -0.2020,  0.0948,
          0.2398,  0.2182, -0.1860, -0.2572],
        [ 0.0039,  0.3405, -0.2087, -0.7660, -0.3993,  1.1604,  0.1000,  0.4489,
          0.0557,  0.1234, -0.6647,  0.1982,  0.2854,  0.2351, -0.5877,  0.0472,
          0.1259,  0.5086, -0.2439, -0.1145]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2080, 0.0040, 0.0000, 0.0000, 1.0676, 0.0000, 0.5721, 0.1048,
         0.3157, 0.0000, 0.1254, 0.2161, 0.0000, 0.0000, 0.1460, 0.0118, 0.3129,
         0.0000, 0.0000],
        [0.0859, 0.0677, 0.0000, 0.0000, 0.0000, 1.1362, 0.0000, 0.0000, 0.1357,
         0.3376, 0.0000, 0.0000, 0.0000, 0.1660, 0.00

In [12]:
# 정의된 순서로 모든 모듈들을 통해 전달

seq_modules = nn.Sequential(
    flatten, 
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
imput_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [13]:
# 마지막 선형 계층은 Softmax모듈에 전달될 logits를 반환
# logits는 모델의 각 분류에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 scale

softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

In [14]:
# parameterize

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0309, -0.0078,  0.0058,  ..., -0.0057, -0.0297,  0.0272],
        [-0.0261, -0.0254, -0.0139,  ..., -0.0165, -0.0292, -0.0203]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0117, -0.0040], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0159,  0.0080, -0.0278,  ..., -0.0045,  0.0200, -0.0434],
        [ 0.0259, -0.0320, -0.0083,  ..., -0.0203,  0.0099, -0.0336]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 